In [ ]:
import pandas as pd
import numpy as  np
import scipy as sc
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

from pandas.errors import ParserError

DESIRED_DATES = ["2007-07-01","2007-10-01","2008-01-01","2008-04-01","2008-07-01","2008-10-01","2009-01-01","2009-04-01","2009-07-01","2009-10-01","2010-01-01","2010-04-01","2010-07-01","2010-10-01","2011-01-01","2011-04-01","2011-07-01","2011-10-01","2012-01-01","2012-04-01","2012-07-01","2012-10-01","2013-01-01","2013-04-01","2013-07-01","2013-10-01","2014-01-01","2014-04-01"]

DESIRED_COUNTRIES = ["Australie","Canada","Hongrie","Norvège","Pologne","Suède","U.K."]

COLORINGS = {"Australie":"red",
             "Canada":"blue",
             "Hongrie":"red",
             "Norvège":"yellow",
             "Pologne":"black",
             "Suède":"blue",
              "U.K.":"purple"}

GROUPS    = {13:["Suède","Canada"],14:["Australie","Hongrie"],
             15:["Pologne"],16:["Norvège"],-1:["U.K."]}

def get_monthly_interest():

    df_monthly_interest =  pd.read_csv("monthly-interest.csv",delimiter="\t",decimal=",").set_index("Date")
    return  df_monthly_interest

def get_monthly_exchange():

    df_monthly_exchange = None

    for year in range(2004,2022):
        for idx, month in enumerate(['January','February','March','April',
                                     'May','June','July','August','September',
                                     'October','November','December']):

            url = "https://www.imf.org/external/np/fin/data/rms_mth.aspx?SelectDate="+str(year)+"-"+\
                    (('0'+str(idx+1)) if (idx+1<10) else  str(idx+1))+ "-"+("29" if year%4==0 and idx+1==2\
                    else ("28" if idx+1==2 else ("30" if idx+1 in [4,6,9,11] else '31')))+"&reportType=REP&tsvflag=Y"

            df = pd.read_csv(url,delimiter="\t",names=range(30),header=None,index_col=0)

            separator_name = 'Representative Exchange Rates for Selected Currencies for '+month+' '+str(year)

            seperator_index = df.index.get_loc(separator_name+' Continued')\
                                if (separator_name+' Continued') in df.index else \
                              df.index.get_loc(separator_name+' (Continued)')

            first_table = df.iloc[:seperator_index]
            second_table = df.iloc[seperator_index:-3]
            monthly_table = None

            for table  in [first_table,second_table]:

                table.columns = table.iloc[1]
                table = table.drop(labels=table.index[:2]).dropna(axis=1,how='all')

                if monthly_table is None:
                    monthly_table = table
                else:
                    monthly_table = pd.concat([monthly_table,table],axis=1)

            first_of_month  = monthly_table.fillna(method='bfill', axis=1).iloc[:, 0]
            timestamp = str(year) + '-' + (('0'+str(idx+1)) if (idx+1<10) else  str(idx+1)) + '-01'
            new_row = pd.DataFrame(data=[[float(value.replace(',','')) if type(value) is str
                                          else value for value  in first_of_month.values]],
                                   index=[timestamp],columns=first_of_month.index.values)

            if df_monthly_exchange is None:
                df_monthly_exchange = new_row
            else:
                df_monthly_exchange = pd.concat([df_monthly_exchange,new_row])

    return df_monthly_exchange

def build_ratios(dataframe):
    df = dataframe.loc[DESIRED_DATES][DESIRED_COUNTRIES]
    df = ((df  - df.shift(periods = 1 ) )/df.shift(periods=1)).iloc[1:]
    return df

def build_delta(dataframe):
    df = dataframe.loc[DESIRED_DATES][DESIRED_COUNTRIES]
    df = (df  - df.shift(periods = 1 ) ).iloc[1:]
    return df

def normalize(np_array):
    dataframe = pd.DataFrame(np_array)
    return ((dataframe-dataframe.mean())/ dataframe.std() ).values

def normalize_by_country(df):
    return (df-df.mean())/df.std()

def regression(interest_input,exchange_input,
               additional_variable_inputs=[]):

    additional_variables = []
    exchange = build_ratios(exchange_input)
    interest = interest_input.loc[DESIRED_DATES][DESIRED_COUNTRIES].iloc[1:]

    for variable_input,build_ratio_flag in additional_variable_inputs:
        variable = build_ratios(variable_input) if build_ratio_flag else \
            variable_input.loc[DESIRED_DATES][DESIRED_COUNTRIES].iloc[1:]
        additional_variables.append(variable)

    average_interest_per_country   = interest.mean(axis=0)
    average_interest_per_timestamp = interest.mean(axis=1)
    average_interest               = average_interest_per_timestamp.mean()

    num_countries                  = len(average_interest_per_country)
    num_timestamps                 = len(average_interest_per_timestamp)

    adjusted_interest  = np.transpose(np.transpose(interest - average_interest_per_country) - (average_interest_per_timestamp - average_interest))

    beta_twfe  =  (adjusted_interest * exchange).mean(axis=1).mean() \
                  / (adjusted_interest * adjusted_interest).mean(axis=1).mean()

    alpha      =  np.array([[1 if desired_country_idx == country_idx else 0 for country_idx in range(num_countries)] +\
                    [1 if desired_timestamp_idx == timestamp_idx else 0 for timestamp_idx in range(num_timestamps)]
                            for desired_country_idx in range(num_countries) for desired_timestamp_idx in range(num_timestamps)])

    flattened_additional_variables = [additional_variable.values.flatten()
                                        for additional_variable in additional_variables]

    if len(additional_variables) != 0:
        adjusted_alpha = []
        for sample_idx, _ in enumerate(alpha):
            adjusted_alpha.append(np.append(alpha[sample_idx],[variables[sample_idx] for variables in    flattened_additional_variables]))
        alpha = adjusted_alpha

    target = exchange.values.flatten() - beta_twfe*interest.values.flatten()

    model = sm.OLS(target,alpha)
    results = model.fit()

    coefficients = results.params

    controlled_exchange = exchange
    for idx, additional_variable in enumerate(additional_variables):
        controlled_exchange = controlled_exchange - coefficients[-(len(additional_variables)-idx)]*additional_variable

    ATT = []
    control_data = controlled_exchange[["U.K."]]
    for treatment_timing, countries in GROUPS.items():
        group_ATT =  []
        group_data = controlled_exchange[countries]
        if treatment_timing != -1 :
            for t in range(0,len(controlled_exchange)):
                if t < treatment_timing:
                    group_ATT.append(0)
                else :
                    group_ATT.append((group_data.iloc[t] - group_data.iloc[treatment_timing-1]).mean() - \
                        (control_data.iloc[t] - control_data.iloc[treatment_timing-1]).mean())

        ATT.append(group_ATT)

    ATT = pd.DataFrame(ATT).transpose()
    ATT.index = DESIRED_DATES[1:]
    ATT.columns = ["Group k=13 (Suède, Canada)",
                   "Group k=14 (Australie, Hongrie)",
                   "Group k=15 (Pologne)",
                   "Group k=16 (Norvège)",
                   "Control (UK)" ]
    ATT = ATT.drop("Control (UK)",axis=1)

    return results, ATT, controlled_exchange


policy_rate      = pd.read_csv("final_data/df_policy_rate.csv",index_col=0)
exchange_rate    = pd.read_csv("final_data/df_exchange_monthly.csv",index_col=0)

binary_treatment = pd.read_csv("final_data/binary_treatment.csv",index_col=0)
balance_of_trade = pd.read_csv("final_data/df_bot.csv",index_col=0)
debt             = pd.read_csv("final_data/df_debt.csv",index_col=0)
growth           = pd.read_csv("final_data/df_growth.csv",index_col=0)
unemployment     = pd.read_csv("final_data/df_unemployment.csv",index_col=0)

additional_variables = [(growth/100+1,False),
                        (balance_of_trade,True),
                        (debt,True),
                        (unemployment,True)]

results, ATT, adjusted_ratios = regression(binary_treatment,
                                  exchange_rate,
                                  additional_variable_inputs=additional_variables)


print("Average Relative Error", np.abs(results.resid).mean())
#print("Average Relative Error", np.abs(results.resid/(exchange_rate.loc[DESIRED_DATES][DESIRED_COUNTRIES].iloc[1:].values.flatten())).mean())
print("Average PValue: ", results.pvalues.mean())

results.summary()

In [ ]:
ATT.plot.line()
plt.savefig("ATT.png")

In [ ]:
ATT.sum().plot.bar()
plt.savefig("ATT-sum.png")

In [ ]:
import matplotlib.pyplot as plt

for treatment_idx, countries in GROUPS.items():
    adjusted_ratios[countries].plot.line(color=['red','blue'])
    if treatment_idx != -1:
        plt.axvline(x=treatment_idx)
        plt.title("Interest rates increased on: " + DESIRED_DATES[treatment_idx])
    else:
        plt.title("Exchange Rates")

plt.savefig("exchange_rates_ajdusted.png")

In [ ]:
import matplotlib.pyplot as plt

for treatment_idx, countries in GROUPS.items():

    build_ratios(exchange_rate)[countries].plot.line(color=['red','blue'])
    plt.title("Interest rates increased on: " + DESIRED_DATES[treatment_idx])

    if treatment_idx != -1:
        plt.axvline(x=treatment_idx)

plt.savefig("exchange_rates.png")